# **Project 1: Integration of Insertion Sort and Merge Sort**

## Team 5

| **Group Member** | **Matriculation Number** |
|---------|-------|
| Jyoshika Barathimogan | |
| Kong Fook Wah | U2421655E |
| Kris Khor Hai Xiang | |

## **Table of Content**

- [Background](#background)
- [Purpose](#purpose-of-project-1)
- [Implementation](#sequence-of-build)
- [Result](#c-analysis-of-time-complexity)



## **Background**

When subarrays become very small, Merge Sort's recursive overhead dominates its cost. In order to improve real-world performance, in practice, we swtich to Insertion Sort below a size threshold S, yielding a hybrid algorithm that reduces constant factors and imrpoves real-world performance. 

## **Purpose of Project 1**

This project aims to determine the optimal threshold value of S that maximises the efficiency of hybrid sort (Insertion Sort & Merge Sort). By applying Insertion Sort when the size of array or sub-array is less than or equal to S, the hybrid sort is expected to have visble improvement in terms of the total number of key comparison required to complete sorting process.

## **Sequence of Build**

### 1. Reproducibility and Constant Variables 
Set up the environment to ensure results are reproducible, including defining all constant variables

### 2. Implementation of Hybrid Algorithm
Develop the bhybrid sorting algorithm that switches to `Insertion Sort` when sub-array size is less than or equal to S

### 3. Generation of Array for Analysis 
Generate test arrays with sizes `n ∈ {1_000 , ... , 10_000_000}`



### Library involved in the experiment

| **Module** | **Purpose** |
|--------|----------|
| `random` | Generate arrays with random numbers |
| `matplotlib` | Visualise the results of the experiment |
| `tqdm` | Show the progress of the algorithm when running |
| `math` | To plot theoretical graph for comparison |


## **1. Reproducibility & Constant Variable**

To maintain consistency across all experiment run, we fix the random environment by setting the seed value of the Python random module to 42. This ensures that the same sequence of random numbers is generated each time, allowing the results to be reproducible and unaffected by external variation.

In [ ]:
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import time
from statistics import median

random.seed(42) #to make sure that the way the number is  generated is consistent 

## **2. Algorithm Implementation**

**Counter class**

In [ ]:
class Counter:
    comparison = 0

**Insertion Sort (in place)**

In [ ]:
def insertionSort(array, low, high, counter: Counter):

    for i in range(low + 1, high):
        key = array[i]
        j = i - 1

        while j >= low:
            counter.comparison += 1

            if array[j] > key:
                array[j + 1] = array[j]
                j -= 1
            else:
                break

        array[j + 1] = key

**Merge Sort *(in place)***

In [ ]:
#aux: auxiliary storage
def merge(array, low, mid, high, aux, counter: Counter):
    
    i, j, k = low, mid, 0
    
    while i < mid and j < high:
        counter.comparison += 1
        
        if array[i] <= array[j]:
            aux[k] = array[i]; i += 1
        else:
            aux[k] = array[j]; j += 1
        k += 1
        
    while i < mid:  
        aux[k] = array[i]
        i += 1
        k += 1
        
    while j < high:   
        aux[k] = array[j]
        j += 1
        k += 1
        
    array[low:high] = aux[:k]


def mergeSort(array, low, high, aux, counter: Counter):
    
    n = high - low
    
    if n <= 1:
        return

    mid = (low + high) // 2

    mergeSort(array, low, mid, aux, counter)
    mergeSort(array, mid, high, aux, counter)
    merge(array, low, mid, high, aux, counter)


**Hybrid Sort *(in place)***

In [ ]:
def hybridSort(array, low, high, S, aux, counter: Counter):
    
    if high - low <= 1:
        return

    if high - low <= S:
        insertionSort(array, low, high, counter)
        return
    
    mid = (low + high) // 2

    hybridSort(array, low, mid, S, aux, counter)
    hybridSort(array, mid, high, S, aux, counter)
    merge(array, low, mid, high, aux, counter)

## Testing Hybrid Sort algorithm

In [ ]:
array = [random.randint(-50, 50) for _ in range(50)]

S = 5

aux = [None] * 50
counter = Counter()

hybridSort(array, 0, 50, S, aux, counter)
print("Array after sorting: ", array)
print("Key Comparison of the array: ", counter.comparison)

It works. Good. Now let's test its efficiency.

## **3. Generating input data**

To avoid the heavy runtime of algorithm due to linear stepping, we sample input sizes on a logarithmic grid with a ratio of around 2 to 2.5 times between consecutive n.

In [ ]:
#Such spacing balance accuracy and runtime while preserving the nlogn trend with far fewer experiments
sizes = [1_000, 2_000, 5_000, 
        10_000, 20_000, 50_000,
        100_000, 200_000, 500_000,
        1_000_000, 2_000_000, 5_000_000,
        10_000_000]

#here we allow the integers in the array to be a range from 0 to 1_000

Now that we have built the Hybrid Sort and the input array, let's analyse its time complexity before we actually test it out.

## **Theoretical Analysis of the Time Complexity**

| **Algorithm**      | **Best Case**     | **Average Case**   | **Worst Case**  |
|----------------|--------------|----------------|--------------|
| Insertion Sort | $O(n)$         | $O(n²)$          | $O(n²)$        |
| Merge Sort     | $O(n log n)$   | $O(n log n)$     | $O(n log n)$   |


## Condition 1: Fixed S, Vary N
### MergeSort
Theoretically, standard Merge Sort keeps splitting until the subarrays are size 1, which continues until $log{_2}{n}$ level. However, in Hybrid Sort, Merge Sort will stop the recursion when subarray $\le S$. (Merge Sort does not go all the way down to level 1)

Then the next question that we are faced with: *How many halvings need to be done until array size shrinks from $n$ to $S$ ?*

Turns out that's when size of array is  $\frac{n}{2^k} \le S$  (See `Image 1` for more information)

Solving for k...

$ \frac{n}{S} \le 2^k $

$log{_2}{\frac{n}{S}} \le k log{_2}{2}$

$k \ge log{_2}{\frac{n}{S}}  $

Therefore we only need about $log{_2}{\frac{n}{S}}$ levels of recursion.

Since $n$ elements are compared at every level in merging process, each level does $ O(n) $ work

Combining both, under the Hybrid Sort, Merge Sort would have a total of $O(n log \frac{n}{S}) $

Merge Sort done, now Insertion Sort.

### Insertion Sort

At last recursion of Merge Sort, Insertion is left with $ \frac{n}{S} $ with each size $ \le S $ to sort.

Insertion Sort at best case runs at $O(n)$ (when the array is mostly sorted). Hence, with Hybrid Sort, we expect the time complexity of Insertion Sort to be $\frac{n}{S} \cdot O(S) = O(n)$

At worst case, Insertion runs at $ O(n^2) $. Hence, with Hybrid Sort, we expect the time complexity of Insertion Sort to be $\frac{n}{S} \cdot O(n^2) = O(n S)$

#### Result

The overall hybrid cost is 

Best Case $ O(n) + O(n log \frac{n}{S}) = O(n + n log \frac{n}{S}) $

Worst Case: $ O(n S) + O(n log \frac{n}{S}) = O(n S +n log \frac{n}{S})$

| **Algorithm**      | **Best Case**     | **Average Case**   | **Worst Case**  |
|----------------|--------------|----------------|--------------|
| Hybrid Sort | $O(n log \frac{n}{S})$         | $O(n S +n log \frac{n}{S})$       | $O(n S +n log \frac{n}{S})$       |




## Condition 2: Fixed N, Vary S

From the previous analysis of time complexity for Merge Sort and Insertion Sort, we notice a pattern.

As S increases,

- Time complexity of Merge Sort *$O(nlog{\frac{n}{S}}) $* shrinks

- But, time complexity of Insertion Sort $O(n S)$ (worst case) grows linearly with S

This shows that the weightage of Insertion Sort and Merge Sort inverted as S increases.

| **Value of S** | **Insertion Sort** | **Merge Sort** | **Total** |
|----------------------------|----------------|------------|--------|
| When S=1 (Pure Merge Sort) | - | $O(nlog{n})$ | $O(nlog{n})$ |
| When S=16 to 64 (Small constant )            | $O(n S) \approx O(n) $ | $O(n log \frac{n}{S}) \approx O(nlogn)$ | $O(nlog{n})$ | 
| When S=$n^\alpha, \alpha>0 $ (Big constant) | $O(n^{1+\alpha})$ | $O(n log \frac{n}{n^{1+\alpha}})$ (Neligible) | $O(n^{1+\alpha})$ (Runtime degrades beyond $O(nlog{n})$)|
| When $S \ge N$ (Reduces to Insertion Sort)|  $O(n)$ (Best Case) <br> $O(n^2)$ (Worst Case) | - | $O(n)$ (Best Case) <br> $O(n^2)$ (Worst Case) |

As $S$ increases, merging gets cheaper but inserting gets more expensive. 

However, there is a sweet spot where both have balance weightage, that is small constant $S$ 

## **Analysis of Time Complexity (Empirical Result)**


### 1. Algorithm Analysis
Conduct performance analysis under two conditions:
**(i)** Fix `S`, vary `n`
**(ii)** Fix `n`, vary `S`

### 2. Comparison of Hybrid Sort with Merge Sort 
Compare the performance of Hybrid Sort against standard Merge Sort at at `n = 10_000_000` using optimal threshold value of S found in (4)

To keep the experiement as fair as possible, we use Mean as an estimate of the expected comparisons over random inputs.

### (i) Fix S, Vary N

In [ ]:
# S value is fixed to be 32
FIXED_VALUE_S = 32
kc_vals = []
median_execution_time = []

for size in tqdm(sizes, desc = "Part 1 (i)"):
    
    #collect comparisons list
    kc_sum = 0
    times = []

    for _ in range(3):

        arr = [random.randint(0, 10**4) for _ in range(size)]
        counter = Counter()
        aux = [None] * size
        start_time = time.perf_counter()

        hybridSort(arr, 0, size, FIXED_VALUE_S, aux, counter)

        end_time = time.perf_counter()
        times.append(end_time - start_time)
        kc_sum += counter.comparison
    
    kc_average = kc_sum / 3
    kc_vals.append(kc_average)

    median_execution_time.append(median(times))

    print("Size of array:", size)
    print("Number of Key Comparison:", kc_average)
    print("Median Execution Time: ", median(times))


#Plot graph
plt.figure(figsize=(10,5))

plt.plot(sizes, kc_vals, marker='o', label='Average Key Comparisons')
theory = [n*S + n*math.log2(n/S) for n in sizes]
plt.plot(sizes, theory, '--', label=r'$O(nS + n\log(n/S))$ reference')

plt.xlabel('Array size (n)')
plt.ylabel('Operation count')
plt.title('Hybrid Sort Complexity')
plt.legend()
plt.grid(True)
plt.show()

### (ii) Fix N, Vary S

In [ ]:
size = 200_000
s_vals = list(range(1, 200))
kc_vals = []
median_execution_time = []
base_arr = [random.randint(0, 10**4) for _ in range(size)]

for s in tqdm(s_vals, desc = "Question part cii"):

    kc_sum = 0
    times = []
    
    for _ in range(3):
        arr = base_arr.copy()
        counter = Counter()
        aux = [None] * size
        start_time = time.perf_counter()

        hybridSort(arr, 0, size, s, aux, counter)

        end_time = time.perf_counter()
        times.append(end_time - start_time)
        kc_sum += counter.comparison
    
    kc_average = kc_sum / 3
    kc_vals.append(kc_average)

    median_execution_time.append(median(times))
    

print("Per-size median times:", median_execution_time)

#plot graph
plt.figure(figsize=(10,5))

plt.plot(s_vals, kc_vals, linestyle='--', label='Key Comparisons vs Threshold S')

plt.xlabel('Threshold S')
plt.ylabel('Key Comparison')
plt.title('Hybrid Sort Complexity with variety of S')
plt.legend()
plt.grid(True)
plt.show()

The graph displays a staircase-like pattern. HMM... is there something wrong with the code? Let's try to understand why we yield such pattern.

## Analysis of Graph of Key Comparison against Threshold S

We know that Merge Sort keep splitting the array until subarray size $\le S$. So at depth $d$ in the recursion tree, each subarray has size $\frac{n}{2^d}$. 

The decision at each depth:

- If $\frac{n}{2^d} \ge S$, we keep calling Merge Sort and split again
- If $\frac{n}{2^d} \le S$, we stop the splitting and call Insertion Sort

Now we imagine there is two consecutive levels that is just before the threshold S (level $d$ and level $d-1$)

- Level $d$: subarray size = $\frac{n}{2^d} = \frac{n}{k}$
- Level $d-1$: subrray size = $\frac{n}{2^{d-1}} = \frac{2n}{k}$

If we pick our threshold value S strictly between these two values ($\frac{2n}{k} < S < \frac{n}{k}$)
- When S is near $\frac{2n}{k}$, the S value becomes too large that MergeSort kept getting called
- When S is near $\frac{n}{k}$, the S value becomes too small that InsertionSort kept getting called

Hence, Insertion Sort will get called at the same level ($\frac{n}{k}$), no matter what exact S we choose in that interval, which explains the *flat region* in our plot


Secondly, if we happen to set $S=\frac{2n}{k}$, 
- Array of size $\frac{2n}{k}$ now satisfy the condition that "$\le S$", hence Insertion Sort will happen one level earlier

Therefore, this explains the *spike* in the graph due to the sudden increase in the number of insertion calls

### Conclusion

As $S$ increases,

- The cutoff moves upward in the recursion tree in jumps 
- And between jumps, the insertion level stays the same, causing the *flat region*
- Finally at the exact multiples of $S=\frac{2n}{k}$, the algorithm shifted an entire level to Insertion Sort, causing the *spike*



### (iii) Determining the optimal value of S

## **(d) Comparison with original MergeSort**

In [ ]:
size = 10_000_000
s_vals = 32

#Hybrid Sort
arr = [random.randint(0, 10**4) for _ in range(size)]
counter = Counter()
aux = [None] * size
start_time = time.perf_counter()

hybridSort(arr, 0, size, s_vals, aux, counter)

end_time = time.perf_counter()

print('Time taken for Hybrid Sort to complete is: ', end_time - start_time)
print("Total number of Key Comparison for Hybrid Sort: ", counter.comparison)

#Merge Sort
arr = [random.randint(0, 10**4) for _ in range(size)]
counter = Counter()
aux = [None] * size
start_time = time.perf_counter()

mergeSort(arr, 0, size, aux, counter)

end_time = time.perf_counter()

print('Time taken for Merge Sort to complete is: ', end_time - start_time)
print("Total number of Key Comparison for Merge Sort: ", counter.comparison)

## **Conclusion**